## Explore Data
- Change between old and new snapshot with corresponding messages will reveal items that need to have messages update for 
- Messages with Change type to show if order book has update
- Open messages not in order book
- grouped messages with more than one to show order book relation

In [1]:
import gzip
import json

def messages_parse(data):
    data_with_no_binary = str(data).split("'")[1:-1][0]
    data_list =[i for i in data_with_no_binary.split('\\n') if i != '']
    messages_data = [json.loads(D) for D in data_list]
    return messages_data 

def messages_read(file="../data/coinbase_BTC-USD_20_10_06_000000-010000.json.gz"):
    file_object = gzip.open(file, "r")
    data = file_object.read()
    messages_data = messages_parse(data)
    return messages_data 

def snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json"):
    snaphsot_data = json.loads(open(file,'r').read())
    return snaphsot_data

def messages_filter(messages_data,initial_clob,final_clob):
    messages_data_filtered = [message_dict for message_dict in messages_data if message_dict['sequence'] >= initial_clob['sequence'] and message_dict['sequence'] <= final_clob['sequence']]
    return messages_data_filtered
            



In [2]:
messages_data = messages_read()
initial_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json")
final_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_15.json")
messages_data_filtered = messages_filter(messages_data,initial_clob,final_clob)


## Difference between final and initial

In [3]:
def difference_identify(l1,l2):
    return [subli for subli in l2 if subli not in l1]

def difference_identify_bids_or_asks(initial_clob,final_clob,bids_or_asks):
    l1 = initial_clob[bids_or_asks]
    l2 = final_clob[bids_or_asks]
    difference_list = difference_identify(l1,l2)
    return difference_list

print (len(final_clob['asks']))
difference_list_asks = difference_identify_bids_or_asks(initial_clob,final_clob,'asks')
print (len(difference_list_asks))


12677
180


In [13]:
print (len(final_clob['bids']))
difference_list_bids = difference_identify_bids_or_asks(initial_clob,final_clob,'bids')
print (len(difference_list_bids))

28413
184


In [4]:
print (difference_list_asks[0])

['10792.95', '0.03455826', '0498bfb5-8b93-46ab-b121-64d6b891e0f1']


In [5]:
filter_order_id_message = lambda array,order_id: [D for D in array if 'order_id' in D.keys() and D['order_id'] ==order_id]
filter_order_id_clob = lambda clob,asks_or_bids,order_id: [subli for subli in clob[asks_or_bids] if subli[2] == order_id]



In [6]:
filter_order_id_message(messages_data_filtered,'0498bfb5-8b93-46ab-b121-64d6b891e0f1')

[{'type': 'open',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:15:06.735819Z',
  'sequence': 16713057500,
  'price': '10792.95',
  'order_id': '0498bfb5-8b93-46ab-b121-64d6b891e0f1',
  'remaining_size': '0.03455826'},
 {'type': 'received',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:15:06.735819Z',
  'sequence': 16713057499,
  'order_id': '0498bfb5-8b93-46ab-b121-64d6b891e0f1',
  'order_type': 'limit',
  'size': '0.03455826',
  'price': '10792.95',
  'client_oid': '746f1dfc-58bb-4ea0-b0d9-372cf32172ce'}]

In [15]:
def difference_dict_create(difference_subli,messages_data_filtered):
    order_id = difference_subli[2]
    messages = filter_order_id_message(messages_data_filtered,order_id)
    types_list = [D['type'] for D in messages]
    return {
        'order_id':order_id,
        'messages':messages,
        'message_count':len(messages),
        'has_open':'open' in types_list
    }

def difference_array_create(difference_list,messages_data_filtered):
    return [difference_dict_create(subli,messages_data_filtered) for subli in difference_list]

difference_array = difference_array_create(difference_list_asks,messages_data_filtered)
print (difference_array[0])

{'order_id': '0498bfb5-8b93-46ab-b121-64d6b891e0f1', 'messages': [{'type': 'open', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:15:06.735819Z', 'sequence': 16713057500, 'price': '10792.95', 'order_id': '0498bfb5-8b93-46ab-b121-64d6b891e0f1', 'remaining_size': '0.03455826'}, {'type': 'received', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:15:06.735819Z', 'sequence': 16713057499, 'order_id': '0498bfb5-8b93-46ab-b121-64d6b891e0f1', 'order_type': 'limit', 'size': '0.03455826', 'price': '10792.95', 'client_oid': '746f1dfc-58bb-4ea0-b0d9-372cf32172ce'}], 'message_count': 2, 'has_open': True}


In [20]:

difference_array_bids = difference_array_create(difference_list_bids,messages_data_filtered)
print (difference_array_bids[0])

{'order_id': '642537c6-d66f-4bcc-8a00-6ee2ad422b27', 'messages': [{'type': 'open', 'side': 'buy', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:14:57.859456Z', 'sequence': 16713054133, 'price': '10792.35', 'order_id': '642537c6-d66f-4bcc-8a00-6ee2ad422b27', 'remaining_size': '3.1'}, {'type': 'received', 'side': 'buy', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:14:57.859456Z', 'sequence': 16713054132, 'order_id': '642537c6-d66f-4bcc-8a00-6ee2ad422b27', 'order_type': 'limit', 'size': '3.1', 'price': '10792.35', 'client_oid': '20201006-0000-0000-0002-000000004035'}], 'message_count': 2, 'has_open': True}


In [21]:
print ([D for D in difference_array_bids if not D['has_open']])

[]


In [16]:
print ([D for D in difference_array if not D['has_open']])

[{'order_id': 'c6be4839-3d3b-418e-b91a-b70ca6813d3a', 'messages': [], 'message_count': 0, 'has_open': False}]


In [19]:
print (len([D for D in difference_array if  len(D['messages']) > 2]))

0


In [18]:
print (difference_array[2])

{'order_id': '8871aea3-f344-4b73-829c-509fbbde35e3', 'messages': [{'type': 'open', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:15:05.925908Z', 'sequence': 16713057198, 'price': '10793', 'order_id': '8871aea3-f344-4b73-829c-509fbbde35e3', 'remaining_size': '2'}, {'type': 'received', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:15:05.925908Z', 'sequence': 16713057197, 'order_id': '8871aea3-f344-4b73-829c-509fbbde35e3', 'order_type': 'limit', 'size': '2', 'price': '10793', 'client_oid': 'd1c4711f-cd86-4362-b499-0eea55a6379a'}], 'message_count': 2, 'has_open': True}


## Filter Messages with Order


In [22]:
print (len([D for D in messages_data_filtered if D['type'] not in ['open','received']]))

85161


In [23]:
print ([D for D in messages_data_filtered if D['type'] not in ['open','received']][0])

{'type': 'done', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:00:05.810671Z', 'sequence': 16712804989, 'order_id': 'd98e69bd-aa7c-40ec-8c34-5087c20b86bd', 'reason': 'canceled', 'price': '10863.19', 'remaining_size': '0.04602702'}


In [25]:
filter_order_id_clob(final_clob,'bids','d98e69bd-aa7c-40ec-8c34-5087c20b86bd')

[]

In [ ]:
def order_ids_pull(orders_data_dict,asks_or_bids):
    return [D[2] for D in orders_data_dict[asks_or_bids]]

def messages_identify(messages_data,order_ids):
    return [D for D in messages_data if 'order_id' in D.keys() and D['order_id'] in order_ids]

order_ids = order_ids_pull(final_clob,'asks')
messages_with_orders = messages_identify(messages_data_filtered,order_ids)


In [28]:
print ([D for D in messages_data_filtered if D['type'] in ['change']][0])

IndexError: list index out of range

In [29]:
print ([D for D in messages_data_filtered if D['type'] in ['match']][0])

{'type': 'match', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:00:06.680213Z', 'sequence': 16712805071, 'trade_id': 105017230, 'maker_order_id': 'a80cad71-1631-4e5f-964b-5d61eaa67dc0', 'taker_order_id': 'fe2e3e20-0a4f-46a2-b796-d7c178ae88a8', 'size': '1.1376', 'price': '10798'}


In [30]:
filter_order_id_message(messages_data_filtered,'fe2e3e20-0a4f-46a2-b796-d7c178ae88a8')

[{'type': 'received',
  'side': 'buy',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:06.680213Z',
  'sequence': 16712805070,
  'order_id': 'fe2e3e20-0a4f-46a2-b796-d7c178ae88a8',
  'order_type': 'limit',
  'size': '1.1376',
  'price': '10798',
  'client_oid': 'f4e5a9f7-5d42-4359-9d73-11445a55e9e8'},
 {'type': 'done',
  'side': 'buy',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:06.680213Z',
  'sequence': 16712805072,
  'order_id': 'fe2e3e20-0a4f-46a2-b796-d7c178ae88a8',
  'reason': 'filled',
  'price': '10798',
  'remaining_size': '0'}]

In [33]:
filter_order_id_clob(initial_clob,'asks','fe2e3e20-0a4f-46a2-b796-d7c178ae88a8')

[]

received 
opened then add to the order book of asks and bids
if type done, then remove from the order book

In [34]:
print (len([D for D in messages_data_filtered if D['type'] in ['open']]))

83274


## open messages not in order book to show that those messages have a done stage after them

In [35]:
print ([D for D in messages_data_filtered if D['type'] in ['open']][0])

{'type': 'open', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:00:06.016022Z', 'sequence': 16712805023, 'price': '10849.9', 'order_id': '58523820-dafc-41cf-a7ab-a88427bbc4d0', 'remaining_size': '0.0460834'}


In [36]:
filter_order_id_clob(initial_clob,'asks','58523820-dafc-41cf-a7ab-a88427bbc4d0')

[]

In [37]:
filter_order_id_message(messages_data_filtered,'58523820-dafc-41cf-a7ab-a88427bbc4d0')

[{'type': 'open',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:06.016022Z',
  'sequence': 16712805023,
  'price': '10849.9',
  'order_id': '58523820-dafc-41cf-a7ab-a88427bbc4d0',
  'remaining_size': '0.0460834'},
 {'type': 'received',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:06.016022Z',
  'sequence': 16712805022,
  'order_id': '58523820-dafc-41cf-a7ab-a88427bbc4d0',
  'order_type': 'limit',
  'size': '0.0460834',
  'price': '10849.9',
  'client_oid': '63430ff0-e280-4e85-8e26-eaaefdecd107'},
 {'type': 'done',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:07.810902Z',
  'sequence': 16712805421,
  'order_id': '58523820-dafc-41cf-a7ab-a88427bbc4d0',
  'reason': 'canceled',
  'price': '10849.9',
  'remaining_size': '0.0460834'}]

In [38]:
print (len([D for D in messages_data_filtered if D['side'] not in ['sell']]))

135777
